# Compression Techniques

This notebook figures out what compression techniques are best suited for the database. It tries a varity of methods including csv, pickle and hdf. Based on these findings the compression technique is chosen. Here, the most important thing is **file size** given that every time someone access the database he is required to download the data file (unless stored locally). Therefore, I do not only test how long it takes for the files to get read in locally but also how long it would take to do so remotely.

It uses the methodology as described here: https://towardsdatascience.com/still-saving-your-data-in-csv-try-these-other-options-9abe8b83db3a

___
The conclusion is that **Pickle (xz)** results in the most efficient loading. While it does have a significant write time, that is not a big deal since GitHub Actions does the conversion.
___

In [1]:
import financedatabase as fd
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
filename = 'compression/equities'
equities = fd.Equities()
df = equities.data

In [3]:
df_results = pd.DataFrame(columns=
    ['method', 'file_size', 'write_time', 'read_time'])
display(df_results)

,method,file_size,write_time,read_time


## CSV

In [4]:
def add_result(df_results, method, file_size, 
               write_time, read_time):
    row = {
        'method': method,
        'file_size': file_size, 
        'write_time': write_time, 
        'read_time':  read_time   
      }
    return pd.concat([df_results, pd.DataFrame([row])], axis=0, ignore_index = True)

In [5]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_csv(filename + '_csv.csv')
#---get the size of file---
filesize = os.path.getsize(filename + '_csv.csv') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(filename + '_csv.csv')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

1.2 s ± 9.24 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
637 ms ± 3.11 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.3169,1.19862,0.636521


In [6]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_csv(filename + 'no_index_csv.csv', \
                                           index=False)
#---get the size of file---
filesize = os.path.getsize(filename + '_no_index_csv.csv') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(filename + 'no_index_csv.csv')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

1.17 s ± 17.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
608 ms ± 6.45 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.31690,1.198620,0.636521
1,CSV No Index,67.99397,1.172448,0.608208


In [7]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_csv(filename + 'no_index_csv.gzip', \
                                           index=False, \
                                           compression='gzip')
#---get the size of file---
filesize = os.path.getsize(filename + '_no_index_csv.gzip') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(filename + 'no_index_csv.gzip', \
                                             compression='gzip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index (GZIP)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

3.14 s ± 30.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
830 ms ± 8.81 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.31690,1.198620,0.636521
1,CSV No Index,67.99397,1.172448,0.608208
2,CSV No Index (GZIP),16.48877,3.143661,0.830191


In [8]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_csv(filename + 'no_index_csv.bz2', \
                                           index=False, \
                                           compression='bz2')
#---get the size of file---
filesize = os.path.getsize(filename + '_no_index_csv.bz2') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(filename + 'no_index_csv.bz2', \
                                             compression='bz2')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index (BZ2)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

6.12 s ± 58.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
2.07 s ± 108 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585


## Pickle

### Base

In [9]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_pickle(filename + '.pkl')
#---get the size of file---
filesize = os.path.getsize(filename + '.pkl') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(filename + '.pkl')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

112 ms ± 2.99 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
137 ms ± 1.6 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508


### GZIP

In [10]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_pickle(filename + '_gzip.pkl', \
                                              compression='gzip')
#---get the size of file---
filesize = os.path.getsize(filename + '_gzip.pkl') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(filename + '_gzip.pkl', \
                                                compression='gzip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (GZIP)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

3.74 s ± 43.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
330 ms ± 1.98 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508
5,Pickle (GZIP),14.331237,3.735072,0.330075


### BZ2

In [11]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_pickle(filename + '_bz2.pkl', \
                                              compression='bz2')
#---get the size of file---
filesize = os.path.getsize(filename + '_bz2.pkl') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(filename + '_bz2.pkl', \
                                                compression='bz2')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (BZ2)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

7.17 s ± 35.2 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
1.25 s ± 15.3 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508
5,Pickle (GZIP),14.331237,3.735072,0.330075
6,Pickle (BZ2),10.235255,7.170931,1.253400


### ZIP

In [12]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_pickle(filename + '_zip.pkl', \
                                              compression='zip')
#---get the size of file---
filesize = os.path.getsize(filename + '_zip.pkl') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(filename + '_zip.pkl', \
                                                compression='zip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (ZIP)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

1.49 s ± 10.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
346 ms ± 1.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508
5,Pickle (GZIP),14.331237,3.735072,0.330075
6,Pickle (BZ2),10.235255,7.170931,1.253400
7,Pickle (ZIP),14.490427,1.492701,0.345828


### XZ

In [13]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_pickle(filename + '_xz.pkl', \
                                              compression='xz')
#---get the size of file---
filesize = os.path.getsize(filename + '_xz.pkl') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(filename + '_xz.pkl', \
                                                compression='xz')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (xz)',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

22.1 s ± 756 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
682 ms ± 7.35 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508
5,Pickle (GZIP),14.331237,3.735072,0.330075
6,Pickle (BZ2),10.235255,7.170931,1.253400
7,Pickle (ZIP),14.490427,1.492701,0.345828
8,Pickle (xz),9.435333,22.078919,0.681835


## HDF

In [14]:
#---saving---
result_save = %timeit -n5 -r5 -o df.to_hdf(filename + '.h5', \
                                           key='key', \
                                           mode='w')
#---get the size of file---
filesize = os.path.getsize(filename + '.h5') / 1024**2
#---load---
result_read = %timeit -n5 -r5 -o pd.read_hdf(filename + '.h5', \
                                             key='key', \
                                             mode='r')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'HDF',
                        filesize, 
                        result_save.average, 
                        result_read.average)
df_results

213 ms ± 29.3 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
248 ms ± 8.35 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,file_size,write_time,read_time
0,CSV,69.316900,1.198620,0.636521
1,CSV No Index,67.993970,1.172448,0.608208
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
4,Pickle,56.924569,0.112318,0.136508
5,Pickle (GZIP),14.331237,3.735072,0.330075
6,Pickle (BZ2),10.235255,7.170931,1.253400
7,Pickle (ZIP),14.490427,1.492701,0.345828
8,Pickle (xz),9.435333,22.078919,0.681835
9,HDF,59.208774,0.213282,0.248143


## Results

In [15]:
df_results.sort_values(by='read_time')

,method,file_size,write_time,read_time
4,Pickle,56.924569,0.112318,0.136508
9,HDF,59.208774,0.213282,0.248143
5,Pickle (GZIP),14.331237,3.735072,0.330075
7,Pickle (ZIP),14.490427,1.492701,0.345828
1,CSV No Index,67.993970,1.172448,0.608208
0,CSV,69.316900,1.198620,0.636521
8,Pickle (xz),9.435333,22.078919,0.681835
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
6,Pickle (BZ2),10.235255,7.170931,1.253400
3,CSV No Index (BZ2),11.744003,6.116914,2.072585


In [16]:
df_results.sort_values(by='write_time')

,method,file_size,write_time,read_time
4,Pickle,56.924569,0.112318,0.136508
9,HDF,59.208774,0.213282,0.248143
1,CSV No Index,67.993970,1.172448,0.608208
0,CSV,69.316900,1.198620,0.636521
7,Pickle (ZIP),14.490427,1.492701,0.345828
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
5,Pickle (GZIP),14.331237,3.735072,0.330075
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
6,Pickle (BZ2),10.235255,7.170931,1.253400
8,Pickle (xz),9.435333,22.078919,0.681835


In [17]:
df_results.sort_values(by='file_size')

,method,file_size,write_time,read_time
8,Pickle (xz),9.435333,22.078919,0.681835
6,Pickle (BZ2),10.235255,7.170931,1.253400
3,CSV No Index (BZ2),11.744003,6.116914,2.072585
5,Pickle (GZIP),14.331237,3.735072,0.330075
7,Pickle (ZIP),14.490427,1.492701,0.345828
2,CSV No Index (GZIP),16.488770,3.143661,0.830191
4,Pickle,56.924569,0.112318,0.136508
9,HDF,59.208774,0.213282,0.248143
1,CSV No Index,67.993970,1.172448,0.608208
0,CSV,69.316900,1.198620,0.636521


## Remote Validation

This can only be ran once the files are pushed to GitHub. This is done outside of this Jupyter Notebook. Note that running this part won't work since I picked one and removed the others from the repository and these statistics are merely to explain why I picked the compression method.

In [30]:
repository_path = "https://raw.githubusercontent.com/JerBouma/FinanceDatabase/main/compression/"
filename = "equities"

df_results = pd.DataFrame(columns=
    ['method', 'read_time'])
display(df_results)

,method,read_time


In [31]:
def add_result(df_results, method, read_time):
    row = {
        'method': method,
        'read_time':  read_time   
      }
    return pd.concat([df_results, pd.DataFrame([row])], axis=0, ignore_index = True)

In [32]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(repository_path + filename + '_csv.csv')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV',
                        result_read.average)

df_results

10.3 s ± 1.03 s per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532


In [33]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(repository_path + filename + '_no_index_csv.csv')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index',
                        result_read.average)

df_results

7.36 s ± 644 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340


In [34]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(repository_path + filename + '_no_index_csv.gzip', \
                                             compression='gzip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index (GZIP)', 
                        result_read.average)

df_results

2.47 s ± 116 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006


In [35]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_csv(repository_path + filename + '_no_index_csv.bz2', \
                                             compression='bz2')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'CSV No Index (BZ2)', 
                        result_read.average)

df_results

3.59 s ± 215 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006
3,CSV No Index (BZ2),3.592286


In [36]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(repository_path + filename + '_gzip.pkl', \
                                                compression='gzip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (GZIP)', 
                        result_read.average)

df_results

2.32 s ± 139 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006
3,CSV No Index (BZ2),3.592286
4,Pickle (GZIP),2.319007


In [37]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(repository_path + filename + '_bz2.pkl', \
                                                compression='bz2')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (BZ2)',
                        result_read.average)

df_results

2.82 s ± 82.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006
3,CSV No Index (BZ2),3.592286
4,Pickle (GZIP),2.319007
5,Pickle (BZ2),2.821927


In [38]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(repository_path + filename + '_zip.pkl', \
                                                compression='zip')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (ZIP)',
                        result_read.average)

df_results

2.62 s ± 58.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006
3,CSV No Index (BZ2),3.592286
4,Pickle (GZIP),2.319007
5,Pickle (BZ2),2.821927
6,Pickle (ZIP),2.623271


In [39]:
#---load---
result_read = %timeit -n5 -r5 -o pd.read_pickle(repository_path + filename + '_xz.pkl', \
                                                compression='xz')
#---save the result to the dataframe---
df_results = add_result(df_results,
                        'Pickle (xz)',
                        result_read.average)

df_results

2.24 s ± 144 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


,method,read_time
0,CSV,10.281532
1,CSV No Index,7.362340
2,CSV No Index (GZIP),2.472006
3,CSV No Index (BZ2),3.592286
4,Pickle (GZIP),2.319007
5,Pickle (BZ2),2.821927
6,Pickle (ZIP),2.623271
7,Pickle (xz),2.240822


## Results

In [41]:
df_results.sort_values(by='read_time')

,method,read_time
7,Pickle (xz),2.240822
4,Pickle (GZIP),2.319007
2,CSV No Index (GZIP),2.472006
6,Pickle (ZIP),2.623271
5,Pickle (BZ2),2.821927
3,CSV No Index (BZ2),3.592286
1,CSV No Index,7.362340
0,CSV,10.281532


In [42]:
pd.read_pickle(repository_path + filename + '_xz.pkl', compression='xz')

,short_name,long_name,summary,currency,sector,industry_group,industry,exchange,market,country,state,city,zipcode,website,market_cap
symbol,,,,,,,,,,,,,,,
000002.SZ,CHINA VANKE CO,"China Vanke Co., Ltd.","China Vanke Co., Ltd., together with its subsi...",CNY,Real Estate,Real Estate,Real Estate Management & Development,SHZ,cn_market,China,NaN,Shenzhen,518083,http://www.vanke.com,Large Cap
000004.SZ,SHENZHEN GUOHUA NE,Shenzhen GuoHua Network Security Technology Co...,Shenzhen GuoHua Network Security Technology Co...,CNY,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology,SHZ,cn_market,China,NaN,Shenzhen,518034,http://www.sz000004.cn,Small Cap
000005.SZ,SHN FOUNTAIN CORP,Shenzhen Fountain Corporation,Shenzhen Fountain Corporation engages in real ...,CNY,Real Estate,Real Estate,Real Estate Management & Development,SHZ,cn_market,China,NaN,Shenzhen,518001,http://www.fountain.com.cn,Small Cap
000006.SZ,SHN ZHENYE GROUP,"Shenzhen Zhenye (Group) Co.,Ltd.","Shenzhen Zhenye (Group) Co.,Ltd. engages in th...",CNY,Real Estate,Real Estate,Real Estate Management & Development,SHZ,cn_market,China,NaN,Shenzhen,518008,http://www.zhenye.com,Small Cap
000007.SZ,SHENZHEN QUANXINHA,"Shenzhen Quanxinhao Co., Ltd.","Shenzhen Quanxinhao Co., Ltd. owns and operate...",CNY,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure",SHZ,cn_market,China,NaN,Shenzhen,518031,http://www.sz000007.com,Micro Cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZMS.SG,Commerzbank AG Nam.-Aktien (Sp.,Commerzbank AG,Commerzbank AG provides banking and capital ma...,EUR,Financials,Banks,Banks,STU,dr_market,Germany,NaN,Frankfurt am Main,60311,http://www.commerzbank.com,Mid Cap
ZZVA.BE,NaN,NaN,NaN,EUR,NaN,NaN,NaN,BER,dr_market,NaN,NaN,NaN,NaN,NaN,NaN
ZZVA.DU,NaN,NaN,NaN,EUR,NaN,NaN,NaN,DUS,dr_market,NaN,NaN,NaN,NaN,NaN,NaN


The conclusion is that **Pickle (xz)** results in the most efficient loading. While it does have a significant write time, that is not a big deal since GitHub Actions does the conversion.